In [ ]:
import pandas as pd
import hashlib
from pathlib import Path

# 설정
input_dir = Path("../data/working/before_processing")
output_dir = Path("../data/working/after_processing")
output_dir.mkdir(parents=True, exist_ok=True)

# pandas 옵션
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

# 숫자 컬럼 목록
numeric_cols = ["회계월", "전표번호", "전표행번", "환율", "전표금액", "차변금액", "대변금액", "전표금액기준통화", "차변금액기준통화", "대변금액기준통화"]

def generate_trx_group_hash(group, coa_col: str) -> str:
    def normalize(v, max_len=10):
        if pd.isna(v) or str(v).strip() == "0":
            return None

        s = str(v).strip().replace(",", "").replace(".", "")  # 쉼표와 점 제거

        # 앞에서부터 max_len 자리까지만 사용
        s = s[:max_len]

        return s

    combo = sorted({normalize(v) for v in group[coa_col] if normalize(v)})
    combo_str = "|".join(combo)
    return hashlib.md5(combo_str.encode("utf-8")).hexdigest()


# 모든 파일의 칼럼명 수집
all_columns = set()
for path in sorted(list(input_dir.glob("*.csv"))):
    try:
        df = pd.read_csv(path, nrows=0, dtype=str, encoding='cp949')
    except UnicodeDecodeError:
        df = pd.read_csv(path, nrows=0, dtype=str, encoding='utf-8')
    all_columns.update([col.upper() for col in df.columns.tolist()])


# 📁 폴더 내 모든 .csv 파일 순회
for path in sorted(list(input_dir.glob("*.csv"))):
    print(f"Processing: {path.name}")

    # 1️⃣ 엑셀 읽기
    try:
        df = pd.read_csv(path, dtype=str, header=0, encoding='cp949')
    except UnicodeDecodeError:
        df = pd.read_csv(path, dtype=str, header=0, encoding='utf-8')
    

    # 2️⃣-1 컬럼명 대문자로 변환
    df.columns = [col.upper() for col in df.columns]

    # 2️⃣-2 컬럼명 지정 (all_columns 기준)
    df = df.reindex(columns=all_columns)

    # 2️⃣-3 숫자형 컬럼 변환
    df[numeric_cols] = df[numeric_cols].apply(
        lambda x: pd.to_numeric(x.str.replace(",", "", regex=False), errors="raise")
    )

    # 3️⃣ 해시 생성
    hashes = (
        df.groupby("전표번호")
          .apply(generate_trx_group_hash, coa_col="계정과목코드")
          .reset_index(name="거래유형그룹_해시값")
    )

    # 4️⃣ 해시 병합
    df_with_hashes = df.merge(hashes, on="전표번호", how="left")

    # 5️⃣ 결과 저장 (확장자 교체 → CSV)
    if len(df) == len(df_with_hashes):
        csv_filename = path.with_suffix(".csv").name
        save_path = output_dir / f"with_hash_{csv_filename}"
        df_with_hashes.to_csv(save_path, index=False, encoding="utf-8-sig")
        print(f"✅ Saved: {save_path}")

print("🎉 모든 파일 처리 완료!")


Processing: 2024-01.csv


/tmp/ipykernel_98300/17626446.py:69: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(generate_trx_group_hash, coa_col="계정과목코드")


✅ Saved: ../data/working/after_processing/with_hash_2024-01.csv
Processing: 2024-02.csv


/tmp/ipykernel_98300/17626446.py:69: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(generate_trx_group_hash, coa_col="계정과목코드")


✅ Saved: ../data/working/after_processing/with_hash_2024-02.csv
Processing: 2024-03.csv


...

✅ Saved: ../data/working/after_processing/with_hash_2025-07.csv
Processing: 2025-08.csv


/tmp/ipykernel_98300/17626446.py:69: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(generate_trx_group_hash, coa_col="계정과목코드")


✅ Saved: ../data/working/after_processing/with_hash_2025-08.csv
Processing: 2025-09.csv


/tmp/ipykernel_98300/17626446.py:69: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(generate_trx_group_hash, coa_col="계정과목코드")


✅ Saved: ../data/working/after_processing/with_hash_2025-09.csv
🎉 모든 파일 처리 완료!


In [ ]:
from pathlib import Path
import pandas as pd

input_dir = Path("../data/working/before_processing")

# 결과를 딕셔너리로 저장
file_columns = {}

for path in sorted(input_dir.glob("*.csv")):
    try:
        # 헤더만 읽기 (nrows=0으로도 가능)
        df = pd.read_csv(path, nrows=0, dtype=str, encoding='cp949')
    except UnicodeDecodeError:
        df = pd.read_csv(path, nrows=0, dtype=str, encoding='utf-8')
    
    # 컬럼명 리스트 저장
    file_columns[path.name] = list(df.columns)

# 확인
for fname, cols in file_columns.items():
    print(f"📂 {fname} ({len(cols)} cols): {cols}")


📂 2024-01.csv (32 cols): ['회사코드', '회사명', '전표번호', 'Source', '작성일시', '갱신일시', '작성자ID', '갱신자ID', '전기일자', '회계월', '자동수동구분', '전표적요', '전표행번', '전표통화코드', '기준통화코드', '환율', '차대구분', '전표금액', '차변금액', '대변금액', '전표금액기준통화', '차변금액기준통화', '대변금액기준통화', '부서코드', '부서명', '계정과목코드', '계정과목명', 'REFERENCE_1', 'REFERENCE_2', 'REFERENCE_3', 'REFERENCE_4', 'REFERENCE_5']
📂 2024-02.csv (32 cols): ['회사코드', '회사명', '전표번호', 'Source', '작성일시', '갱신일시', '작성자ID', '갱신자ID', '전기일자', '회계월', '자동수동구분', '전표적요', '전표행번', '전표통화코드', '기준통화코드', '환율', '차대구분', '전표금액', '차변금액', '대변금액', '전표금액기준통화', '차변금액기준통화', '대변금액기준통화', '부서코드', '부서명', '계정과목코드', '계정과목명', 'REFERENCE_1', 'REFERENCE_2', 'REFERENCE_3', 'REFERENCE_4', 'REFERENCE_5']
📂 2024-03.csv (32 cols): ['회사코드', '회사명', '전표번호', 'Source', '작성일시', '갱신일시', '작성자ID', '갱신자ID', '전기일자', '회계월', '자동수동구분', '전표적요', '전표행번', '전표통화코드', '기준통화코드', '환율', '차대구분', '전표금액', '차변금액', '대변금액', '전표금액기준통화', '차변금액기준통화', '대변금액기준통화', '부서코드', '부서명', '계정과목코드', '계정과목명', 'REFERENCE_1', 'REFERENCE_2', 'REFERENCE_3', 'REFERENCE_4', 'REF